In [35]:
# Import required libraries and dependencies
import pandas as pd
import requests
import os
import datetime
import time
import json
import numpy as np

In [3]:
# Create coingecko rest client
from pycoingecko import CoinGeckoAPI
cg = CoinGeckoAPI()

In [4]:
# Get date two weeks from today
start_date_two_weeks_ago = (pd.Timestamp.now() - pd.Timedelta('14 days')).timestamp()
start_date_two_weeks_ago

1662312214.920131

In [5]:
# Get date right now
end_date_today = (pd.Timestamp.now()).timestamp()
end_date_today

1663521815.313822

In [22]:
symbols = ['bitcoin', 'ethereum', 'ripple', 'cardano', 'solana']

# Create logic to loop through symbols and put into data frame
to_merge_list = []
for symbol in symbols:
    df = cg.get_coin_market_chart_range_by_id(id=f'{symbol}', vs_currency='usd', from_timestamp=start_date_two_weeks_ago, to_timestamp=end_date_today)
    df = df['prices']
    df_two_weeks = pd.DataFrame(df)
    df_two_weeks.columns = ['date', 'price']
    df_two_weeks['daily_return'] = df_two_weeks['price'].pct_change().dropna()
    df_daily_return = df_two_weeks['daily_return']
    to_merge_list.append({f'{symbol}': df_daily_return})
to_merge_list[2]

{'ripple': 0           NaN
 1     -0.002673
 2     -0.000762
 3      0.003476
 4     -0.001038
          ...   
 331    0.003041
 332    0.005317
 333   -0.007527
 334    0.008821
 335    0.016371
 Name: daily_return, Length: 336, dtype: float64}

In [34]:
daily_pct = cg.get_price(ids='bitcoin', vs_currencies='usd',include_24hr_change='true')
daily_pct = pd.DataFrame(daily_pct)
daily_pct = daily_pct.T
daily_pct.loc[:, 'usd_24h_change']

bitcoin   -2.844271
Name: usd_24h_change, dtype: float64

In [36]:
np.absolute(daily_pct.loc[:, 'usd_24h_change'])

bitcoin    2.844271
Name: usd_24h_change, dtype: float64

In [37]:
crypto_swing_thresholds = {'Coin':['bitcoin','ethereum','ripple','cardano','solana'],'pct_change_Threshold':[0.029452,0.038992,0.044968,0.045519,0.057073]}

swing_thresholds_df = pd.DataFrame(crypto_swing_thresholds).set_index('Coin')
swing_thresholds_df

,pct_change_Threshold
Coin,
bitcoin,0.029452
ethereum,0.038992
ripple,0.044968
cardano,0.045519
solana,0.057073
